In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA, FastICA
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_squared_error

In [5]:
data = pd.read_excel('BENG 212 PROJECT_with highest_only_numbers_log.xlsx', sheet_name = 'Sheet1', index_col = 0)

data

,Expression dapF_parent,Expression dapF_sup_1-1,Expression dapF_sup_3-1,Expression dgk_parent,Expression dgk_sup_1-1,Expression dgk_sup_2-1,Expression dgk_sup_3-1,Expression entC_parent,Expression entC_sup_1-1,Expression entC_sup_2-1,Expression entC_sup_3-1,Expression ppk,Expression ppk_sup_1-1,Expression ppk_sup_1-2,Expression zwf_parent,Expression zwf_sup_1-1,Expression zwf_sup_1-2
Synonym,,,,,,,,,,,,,,,,,
b0001,0.713561,1.032790,0.515126,-0.279713,-0.652867,0.032790,-0.259722,0.993243,-1.000000,0.280901,0.039893,-0.122656,-0.410522,-0.203670,-0.369695,0.070126,0.248218
b0002,0.419149,-0.169925,0.075921,-0.657577,-1.315355,-0.483233,-0.952482,-0.439522,-3.286322,-0.741387,-0.951532,0.146486,-0.056143,0.463231,0.076962,-0.097655,-0.534791
b0003,0.096560,-1.001784,-0.150221,-0.105407,-1.116964,-0.120837,-0.520445,-0.714770,-2.748193,-0.501848,-2.158230,0.323321,0.081300,0.518401,0.182354,-0.244254,-0.452982
b0004,0.164512,-1.000000,-0.045281,-0.004058,-0.983880,0.047829,-0.470109,-0.530807,-2.708345,-0.508205,-1.242681,0.299560,0.034270,0.542734,0.164512,-0.256565,-0.458925
b0005,-0.106915,-0.106915,-0.347923,0.280108,1.000000,2.421464,0.099536,0.000000,-1.678072,0.857981,-0.192645,-0.152003,0.263034,0.201634,0.000000,0.893085,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b4705,1.639531,2.400240,1.673772,-0.112210,0.400240,1.226592,0.192645,-1.342106,-0.331982,0.421036,-0.984058,0.401617,0.152003,0.103093,-0.292782,-0.177305,0.218180
b4706,0.347923,0.347923,0.125531,-0.652077,-1.459432,-0.874469,-1.874469,-1.362570,-1.362570,-1.847997,0.000000,-1.000000,-1.263034,-0.584963,-0.289507,-0.874469,-0.289507
b4707,-0.064130,-0.131245,-0.131245,0.176878,-0.131245,0.283793,-0.131245,-0.308122,-0.378512,-0.530515,0.823122,0.061401,0.061401,-0.131245,-0.275634,-0.201634,-0.275634


In [4]:
gene_info = pd.read_excel('GSE85914_combined.xlsx', sheet_name = 'COMBINED', sep=r'\s*,\s*', encoding="utf-8-sig", index_col = 2)
gene_info.head()

,Strand,Name,Product,Expression WT,Expression dapF_parent,Expression dapF_sup_1-1,Expression dapF_sup_3-1,Expression WT.1,Expression dgk_parent,Expression dgk_sup_1-1,...,Expression entC_sup_1-1,Expression entC_sup_2-1,Expression entC_sup_3-1,Expression WT.3,Expression ppk,Expression ppk_sup_1-1,Expression ppk_sup_1-2,Expression WT.4,Expression zwf_parent,Expression zwf_sup_1-1
Synonym,,,,,,,,,,,,,,,,,,,,,
b0001,+,thrL,thr operon leader peptide,261,428,534,373,261,215,166,...,107,260,220,319,293,240,277,261,202,274
b0002,+,thrA,Bifunctional aspartokinase/homoserine dehydrog...,1314,1757,1168,1385,1314,833,528,...,201,1173,1014,655,725,630,903,1314,1386,1228
b0003,+,thrB,homoserine kinase,809,865,404,729,809,752,373,...,192,911,289,414,518,438,593,809,918,683
b0004,+,thrC,L-threonine synthase,356,399,178,345,356,355,180,...,84,386,232,208,256,213,303,356,399,298
b0005,+,yaaX,DUF2502 family putative periplasmic protein,14,13,13,11,14,17,28,...,5,29,14,20,18,24,23,14,14,26


In [8]:
[m, n] = data.shape # save these for future formulas. m = #genes, n = #samples

# normalize the data
dataframe = pd.DataFrame(data, 
                         columns = data.columns, index = data.index)

In [12]:
# compute the covariance matrix


# alternative #2: pandas covariance
Cx = dataframe.T.cov()*((n-1)/n) 

In [13]:
# display the 10x10 section
Cx.iloc[:10, :10] #iloc uses numbers instead of indices (which would be loc)

Synonym,b0001,b0002,b0003,b0004,b0005,b0006,b0007,b0008,b0009,b0010
Synonym,,,,,,,,,,
b0001,0.287429,0.241013,0.094191,0.101176,0.052032,0.001296,-0.036410,-0.031139,0.062445,0.184751
b0002,0.241013,0.720486,0.606304,0.573355,0.229176,-0.012331,0.136099,0.015777,0.095240,0.275664
b0003,0.094191,0.606304,0.696815,0.604191,0.288928,0.054284,0.135031,0.038377,0.057512,0.118348
b0004,0.101176,0.573355,0.604191,0.557827,0.278563,-0.007257,0.126812,0.041289,0.030395,0.112185
b0005,0.052032,0.229176,0.288928,0.278563,0.674108,-0.089380,0.104780,0.013830,0.001318,0.056927
b0006,0.001296,-0.012331,0.054284,-0.007257,-0.089380,0.160801,-0.015672,-0.021635,0.020725,0.006388
b0007,-0.036410,0.136099,0.135031,0.126812,0.104780,-0.015672,0.129069,0.019394,0.016745,0.044495
b0008,-0.031139,0.015777,0.038377,0.041289,0.013830,-0.021635,0.019394,0.064741,-0.045243,-0.066444
b0009,0.062445,0.095240,0.057512,0.030395,0.001318,0.020725,0.016745,-0.045243,0.195735,0.215384


In [14]:
# remove the diagonal with subtraction
Cx_offdiag = np.subtract(Cx, np.multiply(Cx, np.identity(m)))

# obtain the top 5 gene pairs
# stack creates a series with the pairs as multi-indexers
# sort puts the highest values first if ascending = False
# [:10] takes the first ten elements, which will contain repeated pairs
# [::2] takes every second element, which eliminates the repeat pairs
top_5_cx = Cx_offdiag.stack().sort_values(ascending = False)[:10][::2]

top_5_cx

Synonym  Synonym
b0278    b3709      24.266061
b0716    b3079      23.958715
b2956    b0532      23.647608
b1476    b0996      23.592003
b1373    b2956      23.370952
dtype: float64

In [16]:
# look up these gene pairs
# create a dataframe to house this info
top_5_info = pd.DataFrame(columns = ['gene1', 'name1', 'product1', 'gene2', 'name2', 'product2',])

# iterate through the pairs. "enumerate" returns a number for easier indexing, too.
for num, gene_pair in enumerate(top_5_cx.index):
    # get information on each gene
    gene_info1 = gene_info.loc[gene_pair[0], ['Name', 'Product']]
    gene_info2 = gene_info.loc[gene_pair[1], ['Name', 'Product']]
    
    # store it in the dataframe we created using a list
    top_5_info.loc[num] =[gene_pair[0]] + gene_info1.tolist() + [gene_pair[1]] + gene_info2.tolist()
    
top_5_info

,gene1,name1,product1,gene2,name2,product2
0,b0278,yagL,CP4-6 prophage; DNA-binding protein,b3709,tnaB,tryptophan transporter of low affinity
1,b0716,ybgO,putative fimbrial protein,b3079,ygjJ,putative periplasmic protein
2,b2956,yggM,DUF1202 family putative secreted protein,b0532,sfmD,putative outer membrane export usher protein
3,b1476,fdnI,"formate dehydrogenase-N, cytochrome B556 (gamm...",b0996,torC,"trimethylamine N-oxide (TMAO) reductase I, cyt..."
4,b1373,tfaR,Rac prophage; putative tail fiber assembly pro...,b2956,yggM,DUF1202 family putative secreted protein


Covarying genes are supposed to perform similar functions and/or be from the same operon. 

    0: yagL is a DNA-binding protein that exists in the cytosol while tnaB resides in the inner membrane. As of now yagL is uncharacterized in EcoCyc and Uniprot. Perhaps the tryptophan transporter is somehow integral yagL or vice versa. 
    
    1: YbgO is a hypothetical protein (according to EcoCyc) but that sequence similarities suggest that it might be an outer membrane porin which could perhaps facilitate the transport of YgjJ from periplasm to outside of the cells (??). Apparently it has been found recently that Ybjo is an inner membrane protein. Perhaps both of these proteins act along the same pathway and are activated and deactivated by the same substrate(s). 
    
    2:Perhaps sfmD is the outer membrane export usher protein which facilitates the secretion of the DUF 1202 family protein, yggM. 
    
    3:TorC is a part of the anaerobic respiratory chain of trimethylamine-N-oxide reductase TorA. Formate dehydrogenase (FdnI) allows E.coli to use formate as major electron donor during anaerobic respiration, when nitrate is used as electron acceptor. I wonder what these two genes have in common.
    
    4: So YggM still has an unknown function and hasn't been characterized while TfaR is part of the tail fiber assembly. I also don't see how these two are related at a cursory glance. 